In [ ]:
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from sklearn.compose import make_column_transformer, make_column_selector, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold

from drn_interactions.config import ExperimentInfo, Config
from drn_interactions.decoding.loaders import FSDecodeDataLoader, FSFastDecodeDataLoader
from drn_interactions.decoding.preprocessors import DecodePreprocessor
from drn_interactions.decoding.encoders import StateEncoder
from drn_interactions.decoding.runners import EncodeRunner
from drn_interactions.decoding.shuffle import shuffle_X


In [ ]:
sessions = ExperimentInfo.foot_shock_sessions_10min
session = sessions[0]
loader = FSDecodeDataLoader(session_name=session, t_start=-600, t_stop=600)
preprocessor = DecodePreprocessor(
    thresh_empty=2,
)
spikes, states = loader()
spikes, states = preprocessor(spikes, states)

# states.name = "block"
ct = make_column_transformer(
    (
        StandardScaler(),
        make_column_selector(dtype_exclude=object)
    ),  
    (
        OneHotEncoder(drop="if_binary"),
        make_column_selector(dtype_include=object)
    ), 
)

estimator = make_pipeline(ct, 
    LGBMRegressor(
        n_estimators=30,
        n_jobs=-1, 
        force_row_wise=True,
        reg_lambda=0.8,
        )
)

estimator = TransformedTargetRegressor(estimator, transformer=StandardScaler())
cv = KFold(shuffle=True)
encoder = StateEncoder(estimator=estimator, cv=cv, verbose=True, combined_mode="combined")
runner = EncodeRunner(
    loader=loader,
    preprocessor=preprocessor,
    encoder=encoder,
)

In [ ]:
pop = runner.run_multiple_pop(sessions=sessions)
pop_shuffle = runner.run_multiple_pop(sessions=sessions, shuffle=True)
state = runner.run_multiple_state(sessions=sessions)
state_shuffle = runner.run_multiple_state(sessions=sessions, shuffle=True)
comb = runner.run_multiple_comb(sessions=sessions, shuffle=False)
comb_shuffle = runner.run_multiple_comb(sessions=sessions, shuffle=True)
limit = runner.run_multiple_limit(sessions=sessions, min_features=1, max_features=30)
dropout = runner.run_multiple_dropout(sessions=sessions)


In [ ]:
dd = Config.derived_data_dir / "encoding"

pop.to_csv(dd / "fs_slow - pop.csv", index=False)
pop_shuffle.to_csv(dd / "fs_slow - pop shuffle.csv", index=False)
comb.to_csv(dd / "fs_slow - comb.csv", index=False)
comb_shuffle.to_csv(dd / "fs_slow - comb shuffle.csv", index=False)
state.to_csv(dd / "fs_slow - state.csv", index=False)
state_shuffle.to_csv(dd / "fs_slow - state shuffle.csv", index=False)
limit.to_csv(dd / "fs_slow - limit.csv", index=False)
dropout.to_csv(dd / "fs_slow - dropout.csv", index=False)

In [ ]:
sessions = ExperimentInfo.foot_shock_sessions_10min
session = sessions[0]
loader = FSFastDecodeDataLoader(session_name=session)
preprocessor = DecodePreprocessor(
    thresh_empty=2,
)
spikes, states = loader()
spikes, states = preprocessor(spikes, states)

ct = make_column_transformer(
    (
        StandardScaler(),
        make_column_selector(dtype_exclude=object)
    ),  
    (
        OneHotEncoder(drop="if_binary"),
        make_column_selector(dtype_include=object)
    ), 
)

estimator = make_pipeline(ct, 
    LGBMRegressor(
        n_estimators=20, 
        n_jobs=-1, 
        force_row_wise=True,
        reg_lambda=0.8,
        )
)

estimator = TransformedTargetRegressor(estimator, transformer=StandardScaler())
cv = KFold(shuffle=True)
encoder = StateEncoder(estimator=estimator, cv=cv, verbose=True)
runner = EncodeRunner(
    loader=loader,
    preprocessor=preprocessor,
    encoder=encoder,
)

In [ ]:
dd = Config.derived_data_dir / "encoding"

pop.to_csv(dd / "fs_fast - pop.csv", index=False)
state.to_csv(dd / "fs_fast - state.csv", index=False)
limit.to_csv(dd / "fs_fast - limit.csv", index=False)
dropout.to_csv(dd / "fs_fast - dropout.csv", index=False)
    